# Keyswitch Analytics

By Kenneth Burchfiel

Released under the MIT license

This notebook demonstrates how to compare results for different keyswitch options. This notebook can easily be modified to compare other elements of your setting (such as your location) instead.

(More documentation to come!)

In [1]:
import pandas as pd
import plotly.express as px
import statsmodels.api as sm
pd.set_option('display.max_columns', 1000)
import numpy as np
import time
start_time = time.time()
analytics_folder = 'Keyswitch_Analytics/' # The location
# where charts and .csv files created in this script
# will be saved.

# Part 1: Data Retrieval

In [2]:
df_results = pd.read_csv('results.csv')
df_results

,Test_Number,Unix_Start_Time,Local_Start_Time,UTC_Start_Time,Characters,Seconds,CPS,WPM,Book,Chapter,Verse_Order,Verse,Verse #,Last 10 Avg,Last 100 Avg,Last 1000 Avg,cumulative_avg,Local_Start_Year,Local_Start_Month,Local_Start_Hour,Count,Local_Start_Date,Local_Start_Minute,Local_Start_10_Minute_Block,Local_Start_15_Minute_Block,Local_Start_30_Minute_Block,Local_End_Time,Local_End_Year,Local_End_Month,Local_End_Date,Local_End_Hour,Local_End_Minute,Local_End_15_Minute_Block,Local_End_10_Minute_Block,Local_End_30_Minute_Block,Mistake_Free_Test,WPM_Rank,WPM_Percentile,Backspaces as % of Verse Length,Incorrect Characters as % of Verse Length,Min Character Time (ms),Median Character Time (ms),Max Character Time (ms),Backspace % Last 10 Avg,Backspace % Last 100 Avg,Backspace % Last 1000 Avg,Session,Test_#_Within_Session,Incorrect Character % Last 10 Avg,Incorrect Character % Last 100 Avg,Incorrect Character % Last 1000 Avg,Test Version,Location,Keyboard,Layout,Caffeine,Custom_1,Custom_2,Custom_3,Autostart,% of Words Typed Correctly,Extra Keystrokes %,Extra Keystrokes % Last 10 Avg,Extra Keystrokes % Last 100 Avg,Extra Keystrokes % Last 1000 Avg
0,1,1.698121e+09,2023-10-24 00:17:38.017017,2023-10-24 04:17:38.017017+00:00,56,6.571754,8.521317,102.255799,GEN,1,1,"In the beginning, God created the heavens and ...",NaN,NaN,NaN,NaN,102.256,2023,10,0,1,2023-10-24,17,2,2,1,2023-10-24 00:17:44.588771456,2023,10,2023-10-24,0,17,2,2,1,NaN,9065,7.169193,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1.698121e+09,2023-10-24 00:18:06.165502,2023-10-24 04:18:06.165502+00:00,135,13.997536,9.644554,115.734653,GEN,1,2,The earth was formless and empty. Darkness was...,NaN,NaN,NaN,NaN,108.995,2023,10,0,1,2023-10-24,18,2,2,1,2023-10-24 00:18:20.163038180,2023,10,2023-10-24,0,18,2,2,1,NaN,7815,19.971323,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1.698121e+09,2023-10-24 00:18:24.381290,2023-10-24 04:18:24.381290+00:00,56,4.714291,11.878774,142.545292,GEN,1,1,"In the beginning, God created the heavens and ...",NaN,NaN,NaN,NaN,120.179,2023,10,0,1,2023-10-24,18,2,2,1,2023-10-24 00:18:29.095581096,2023,10,2023-10-24,0,18,2,2,1,NaN,3338,65.823433,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1.698121e+09,2023-10-24 00:21:24.331389,2023-10-24 04:21:24.331389+00:00,56,4.878704,11.478458,137.741497,GEN,1,1,"In the beginning, God created the heavens and ...",NaN,NaN,NaN,NaN,124.569,2023,10,0,1,2023-10-24,21,3,2,1,2023-10-24 00:21:29.210093071,2023,10,2023-10-24,0,21,2,3,1,NaN,4187,57.128226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1.698121e+09,2023-10-24 00:22:03.749152,2023-10-24 04:22:03.749152+00:00,135,19.078445,7.076048,84.912580,GEN,1,2,The earth was formless and empty. Darkness was...,NaN,NaN,NaN,NaN,116.638,2023,10,0,1,2023-10-24,22,3,2,1,2023-10-24 00:22:22.827597200,2023,10,2023-10-24,0,22,2,3,1,NaN,9629,1.392872,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9759,9760,1.710602e+09,2024-03-16 11:17:41.671556,2024-03-16 15:17:41.671556+00:00,78,6.773794,11.514964,138.179570,JHN,9,30758,"Jesus said to him, ""You have both seen him, an...",37.0,135.626543,145.029403,133.586009,133.377,2024,3,11,1,2024-03-16,17,2,2,1,2024-03-16 11:17:48.445350400,2024,3,2024-03-16,11,17,2,2,1,0.0,4105,57.968046,2.564103,7.692308,NaN,NaN,NaN,NaN,NaN,NaN,209,8,6.673974,5.184698,6.715065,v2,home,cherry mx2a brown,dvorak,yes,NaN,NaN,NaN,0.0,94.117647,14.102564,12.015566,8.909651,NaN
9760,9761,1.710602e+09,2024-03-

The following cell stores a list of start and end times for each trial. 

In [3]:
trial_start_and_end_times_list = [('2024-03-11 11:25', '2024-03-11 11:56'),
                                  ('2024-03-16 10:40', '2024-03-16 11:19')]


The next cell uses a list comprehension to create a filtered copy of df_results that includes only results that appear *after* the first timestamp in the tuple and *before* the second timestamp. (In order words, results that match the timestamps in the list exactly won't get included.)

In [4]:
df_trial_results = pd.concat(
[df_results.query(f"Local_Start_Time > '{time_pair[0]}' \
& Local_Start_Time < '{time_pair[1]}'").copy() for time_pair 
in trial_start_and_end_times_list]) # This code, for each set of timestamps
# within trial_start_and_end_times_list, creates a copy of df_results
# that only includes results after the first timestamp and before the second 
# one. It next combines these copies together via pd.concat().
df_trial_results

,Test_Number,Unix_Start_Time,Local_Start_Time,UTC_Start_Time,Characters,Seconds,CPS,WPM,Book,Chapter,Verse_Order,Verse,Verse #,Last 10 Avg,Last 100 Avg,Last 1000 Avg,cumulative_avg,Local_Start_Year,Local_Start_Month,Local_Start_Hour,Count,Local_Start_Date,Local_Start_Minute,Local_Start_10_Minute_Block,Local_Start_15_Minute_Block,Local_Start_30_Minute_Block,Local_End_Time,Local_End_Year,Local_End_Month,Local_End_Date,Local_End_Hour,Local_End_Minute,Local_End_15_Minute_Block,Local_End_10_Minute_Block,Local_End_30_Minute_Block,Mistake_Free_Test,WPM_Rank,WPM_Percentile,Backspaces as % of Verse Length,Incorrect Characters as % of Verse Length,Min Character Time (ms),Median Character Time (ms),Max Character Time (ms),Backspace % Last 10 Avg,Backspace % Last 100 Avg,Backspace % Last 1000 Avg,Session,Test_#_Within_Session,Incorrect Character % Last 10 Avg,Incorrect Character % Last 100 Avg,Incorrect Character % Last 1000 Avg,Test Version,Location,Keyboard,Layout,Caffeine,Custom_1,Custom_2,Custom_3,Autostart,% of Words Typed Correctly,Extra Keystrokes %,Extra Keystrokes % Last 10 Avg,Extra Keystrokes % Last 100 Avg,Extra Keystrokes % Last 1000 Avg
9335,9336,1.710171e+09,2024-03-11 11:26:37.472366,2024-03-11 15:26:37.472366+00:00,192,16.778450,11.443250,137.319004,JHN,5,30510,"Jesus therefore answered them, ""Most certainly...",19.0,137.585592,138.500531,128.903824,133.048,2024,3,11,1,2024-03-11,26,3,2,1,2024-03-11 11:26:54.250815700,2024,3,2024-03-11,11,26,2,3,1,0.0,4263,56.349857,3.125000,8.333333,NaN,NaN,NaN,NaN,NaN,NaN,203,1,6.676314,5.521322,7.532080,v2,home,cherry red,dvorak,yes,NaN,NaN,NaN,0.0,88.235294,13.541667,12.194104,NaN,NaN
9336,9337,1.710171e+09,2024-03-11 11:26:56.220335,2024-03-11 15:26:56.220335+00:00,152,11.056986,13.746965,164.963585,JHN,5,30511,"For the Father has affection for the Son, and ...",20.0,140.528065,138.713826,128.928716,133.052,2024,3,11,1,2024-03-11,26,3,2,1,2024-03-11 11:27:07.277320700,2024,3,2024-03-11,11,27,2,3,1,0.0,611,93.752560,1.315789,4.605263,NaN,NaN,NaN,NaN,NaN,NaN,203,2,6.901546,5.540529,7.531622,v2,home,cherry red,dvorak,yes,NaN,NaN,NaN,0.0,96.551724,5.921053,12.315621,NaN,NaN
9337,9338,1.710171e+09,2024-03-11 11:27:11.907570,2024-03-11 15:27:11.907570+00:00,106,7.391912,14.339997,172.079964,JHN,5,30512,For as the Father raises the dead and gives th...,21.0,142.400652,138.888181,128.988686,133.056,2024,3,11,1,2024-03-11,27,3,2,1,2024-03-11 11:27:19.299482300,2024,3,2024-03-11,11,27,2,3,1,1.0,268,97.265465,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,203,3,6.300453,5.540529,7.515985,v2,home,cherry red,dvorak,yes,NaN,NaN,NaN,0.0,100.000000,0.000000,11.168080,NaN,NaN
9338,9339,1.710171e+09,2024-03-11 11:27:21.423686,2024-03-11 15:27:21.423686+00:00,71,6.279210,11.307155,135.685859,JHN,5,30513,"For the Father judges no one, but he has given...",22.0,143.358556,138.959112,128.984768,133.056,2024,3,11,1,2024-03-11,27,3,2,1,2024-03-11 11:27:27.702896000,2024,3,2024-03-11,11,27,2,3,1,0.0,4550,53.410488,1.408451,1.408451,NaN,NaN,NaN,NaN,NaN,NaN,203,4,5.989321,5.501583,7.511974,v2,home,cherry red,dvorak,yes,NaN,NaN,NaN,0.0,93.333333,2.816901,10.376323,NaN,NaN
9339,9340,1.710171e+09,2024-03-11 11:27:31.307316,2024-03-11 15:27:31.307316+00:00,126,10.224435,12.323420,147.881036,JHN,5,30514,"that all may honor the Son, even as they honor...",23.0,147.057733,139.134097,128.998703,133.058,2024,3,11,1,2024-03-11,27,3,2,1,2024-03-11 11:27:41.531750700,2024,3,2024-03-11,11,27,2,3,1,0.0,2442,75.000000,1.587302,4.761905,NaN,NaN,NaN,NaN,NaN,NaN,203,5,5.024833,5.513488,7.514355,v2,home,cherry red,dvorak,yes,NaN,NaN,NaN,0.0,96.000000,9.523810,8.701586,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9759,9760,1.710602e+09,2024-03-16 11:17:41.671556,2024-03-16 15:17:41.671556+00:00,78,6.773794,11.5

In [5]:
df_trial_results.to_csv(analytics_folder+'trial_results.csv', 
index = False)

Here's an alternative version of the previous cell that uses a for loop rather than a list comprehension. It's a bit easier to interpret but requires more lines of code, so I prefer the previous version.

In [6]:
# trial_results_df_list = []
# for time_pair in trial_start_and_end_times_list:
#     df_selected_results = df_results.query(
#         f"Local_Start_Time > '{time_pair[0]}' \
# & Local_Start_Time < '{time_pair[1]}'").copy()
#     trial_results_df_list.append(df_selected_results)

# df_trial_results = pd.concat([df for df in trial_results_df_list])
# df_trial_results

In [7]:
df_mistake_free_trial_results = df_trial_results.query(
"Mistake_Free_Test == 1").copy().sort_values(
'WPM', ascending = False).reset_index(drop=True)
# The following line will serve as the set of x values within a scatter plot.
# The use of the modulo operator allows sequential values to be spaced apart
# while still allowing for a relatively narrow graph.
df_mistake_free_trial_results['Scatter_X_Values'] = (
df_mistake_free_trial_results.index % 5)
df_mistake_free_trial_results

,Test_Number,Unix_Start_Time,Local_Start_Time,UTC_Start_Time,Characters,Seconds,CPS,WPM,Book,Chapter,Verse_Order,Verse,Verse #,Last 10 Avg,Last 100 Avg,Last 1000 Avg,cumulative_avg,Local_Start_Year,Local_Start_Month,Local_Start_Hour,Count,Local_Start_Date,Local_Start_Minute,Local_Start_10_Minute_Block,Local_Start_15_Minute_Block,Local_Start_30_Minute_Block,Local_End_Time,Local_End_Year,Local_End_Month,Local_End_Date,Local_End_Hour,Local_End_Minute,Local_End_15_Minute_Block,Local_End_10_Minute_Block,Local_End_30_Minute_Block,Mistake_Free_Test,WPM_Rank,WPM_Percentile,Backspaces as % of Verse Length,Incorrect Characters as % of Verse Length,Min Character Time (ms),Median Character Time (ms),Max Character Time (ms),Backspace % Last 10 Avg,Backspace % Last 100 Avg,Backspace % Last 1000 Avg,Session,Test_#_Within_Session,Incorrect Character % Last 10 Avg,Incorrect Character % Last 100 Avg,Incorrect Character % Last 1000 Avg,Test Version,Location,Keyboard,Layout,Caffeine,Custom_1,Custom_2,Custom_3,Autostart,% of Words Typed Correctly,Extra Keystrokes %,Extra Keystrokes % Last 10 Avg,Extra Keystrokes % Last 100 Avg,Extra Keystrokes % Last 1000 Avg,Scatter_X_Values
0,9694,1.710601e+09,2024-03-16 10:52:20.590762,2024-03-16 14:52:20.590762+00:00,62,3.783738,16.385914,196.630971,JHN,8,30689,They didn't understand that he spoke to them a...,27.0,164.170065,142.203054,132.313095,133.321,2024,3,10,1,2024-03-16,52,6,4,2,2024-03-16 10:52:24.374499600,2024,3,2024-03-16,10,52,4,6,2,1.0,8,99.928308,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,208,50,2.671840,4.587106,6.850653,v2,home,cherry mx2a black,dvorak,yes,NaN,NaN,NaN,0.0,100.0,0.0,3.671861,8.153507,NaN,0
1,9685,1.710601e+09,2024-03-16 10:50:34.994189,2024-03-16 14:50:34.994189+00:00,84,5.304726,15.834936,190.019236,JHN,8,30680,"I am one who testifies about myself, and the F...",18.0,148.974096,140.015417,132.010653,133.295,2024,3,10,1,2024-03-16,50,6,4,2,2024-03-16 10:50:40.298915100,2024,3,2024-03-16,10,50,4,6,2,1.0,16,99.846374,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,208,41,3.413567,4.716992,6.883005,v2,home,cherry mx2a black,dvorak,yes,NaN,NaN,NaN,0.0,100.0,0.0,5.228614,8.522203,NaN,1
2,9349,1.710171e+09,2024-03-11 11:30:12.296669,2024-03-11 15:30:12.296669+00:00,100,6.586644,15.182239,182.186865,JHN,5,30523,It is another who testifies about me. I know t...,32.0,148.742679,139.995001,129.153923,133.073,2024,3,11,1,2024-03-11,30,4,3,2,2024-03-11 11:30:18.883312900,2024,3,2024-03-11,11,30,3,4,2,1.0,46,99.539123,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,203,14,3.738254,5.286069,7.460851,v2,home,cherry mx2a brown,dvorak,yes,NaN,NaN,NaN,0.0,100.0,0.0,7.207516,NaN,NaN,2
3,9663,1.710600e+09,2024-03-16 10:45:34.934465,2024-03-16 14:45:34.934465+00:00,56,3.704110,15.118342,181.420103,JHN,7,30658,But this multitude that doesn't know the law i...,49.0,146.595914,136.034084,131.467590,133.263,2024,3,10,1,2024-03-16,45,5,4,2,2024-03-16 10:45:38.638574900,2024,3,2024-03-16,10,45,4,5,2,1.0,57,99.426465,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,208,19,4.326600,5.478075,6.929890,v2,home,cherry mx2a black,dvorak,yes,NaN,NaN,NaN,0.0,100.0,0.0,8.941120,9.919648,NaN,3
4,9693,1.710601e+09,2024-03-16 10:52:07.491163,2024-03-16 14:52:07.491163+00:00,155,10.501553,14.759721,177.116657,JHN,8,30688,I have many things to speak and to judge conce...,26.0,157.376027,141.505103,132.232886,133.314,2024,3,10,1,2024-03-16,52,6,4,2,2024-03-16 10:52:17.992716200,2024,3,2024-03-16,10,52,4,6,2,1.0,119,98.791479,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,208,49,3.782952,4.668739,6.863948,v2,home,cherry mx2a black,dvorak,yes,NaN,NaN,NaN,0.0,100.0,0.0,5.338528,8.296364,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,9682,1.710601e+09,2024-03-16 10:49:27.328572,2024-03-16 14:49:27.328572+00:00,49,4.034740,12.144524,145.734287,JHN,8,30677,You judge accordi

In [8]:
df_mistake_free_trial_results.to_csv(
analytics_folder+'mistake_free_trial_results.csv', index = False)

# Part 2: Descriptive Statistics and Visualizations

In [9]:
keyboard_color_discrete_map ={'cherry red':'#EE0000', 
'cherry mx2a brown':'#844204', 'cherry mx2a black':'black'}

# I found the RGB color code for the brown shade by visiting
# https://upload.wikimedia.org/wikipedia/commons/0/0e/Color_icon_brown_v2.svg
# (a public-domain file); downloading the image; and opening it within 
# my code editor. This allowed me to view the RGB values for the 9 brown 
# colors that comprise the square.

In [10]:
df_wpm_and_accuracy_by_keyboard = (
df_trial_results.pivot_table(
index = 'Keyboard', 
values = ['WPM', 'Extra Keystrokes %'], 
aggfunc = 'mean').reset_index(
).sort_values('WPM', ascending = False).rename(
columns = {'WPM':'Mean WPM', 
'Extra Keystrokes %':'Mean Extra Keystrokes %'}))
df_wpm_and_accuracy_by_keyboard['Category'] = 'All'
df_wpm_and_accuracy_by_keyboard

,Keyboard,Mean Extra Keystrokes %,Mean WPM,Category
0,cherry mx2a black,7.822162,146.128523,All
1,cherry mx2a brown,11.053259,142.747753,All
2,cherry red,9.144435,142.154919,All


In [11]:
df_wpm_and_accuracy_by_keyboard_and_mistake_free_status = (
df_trial_results.pivot_table(
index = ['Keyboard','Mistake_Free_Test'], 
values = ['WPM', 'Extra Keystrokes %'], 
aggfunc = 'mean').reset_index(
).sort_values('WPM', ascending = False).rename(
columns = {'WPM':'Mean WPM', 
'Extra Keystrokes %':'Mean Extra Keystrokes %'}))
df_wpm_and_accuracy_by_keyboard_and_mistake_free_status[
'Category'] = np.where(
df_wpm_and_accuracy_by_keyboard_and_mistake_free_status[
'Mistake_Free_Test'] == 1, 'No Mistakes', 'Mistakes')
df_wpm_and_accuracy_by_keyboard_and_mistake_free_status


,Keyboard,Mistake_Free_Test,Mean Extra Keystrokes %,Mean WPM,Category
1,cherry mx2a black,1.0,0.000000,167.206949,No Mistakes
3,cherry mx2a brown,1.0,0.000000,165.170891,No Mistakes
5,cherry red,1.0,0.000000,161.835953,No Mistakes
4,cherry red,0.0,11.430544,137.234661,Mistakes
0,cherry mx2a black,0.0,11.588388,135.979651,Mistakes
2,cherry mx2a brown,0.0,14.440548,135.876146,Mistakes


In [12]:
df_wpm_accuracy_comparison = pd.concat([
df_wpm_and_accuracy_by_keyboard,
df_wpm_and_accuracy_by_keyboard_and_mistake_free_status])
df_wpm_accuracy_comparison.to_csv(
analytics_folder+'wpm_accuracy_pivot.csv', index = False)

In [13]:
fig_wpm_comparison = px.bar(df_wpm_accuracy_comparison, 
x = 'Category', y = 'Mean WPM', 
color = 'Keyboard', text_auto='.3f',
barmode = 'group',
color_discrete_map=keyboard_color_discrete_map)
fig_wpm_comparison.write_html(analytics_folder+'wpm_comparison.html')
fig_wpm_comparison

In [14]:
fig_accuracy_comparison = px.bar(
df_wpm_accuracy_comparison.query("Category != 'No Mistakes'"), 
x = 'Category', y = 'Mean Extra Keystrokes %', 
color = 'Keyboard', text_auto='.3f',
barmode = 'group',
color_discrete_map=keyboard_color_discrete_map)
fig_accuracy_comparison.write_html(
analytics_folder+'accuracy_comparison.html')
fig_accuracy_comparison

In [15]:
fig_keyswitch_accuracy_and_wpm_comparison = px.scatter(df_trial_results, x= 'Extra Keystrokes %', 
y = 'WPM', trendline = 'ols', color = 'Keyboard', 
color_discrete_map = keyboard_color_discrete_map)
fig_keyswitch_accuracy_and_wpm_comparison.write_html(analytics_folder+'keyswitch_accuracy_and_wpm_comparison.html')
fig_keyswitch_accuracy_and_wpm_comparison.write_html(
analytics_folder+'accuracy_wpm_comparison.html')
fig_keyswitch_accuracy_and_wpm_comparison

In [16]:
fig_mistake_free_wpm_by_keyboard = px.scatter(
df_mistake_free_trial_results, 
x = 'Scatter_X_Values', y = 'WPM', color = 'Keyboard', 
color_discrete_map=keyboard_color_discrete_map, width = 300, height = 500)
fig_mistake_free_wpm_by_keyboard.update_xaxes(range = [-1, 5])
fig_mistake_free_wpm_by_keyboard.write_html(
analytics_folder+'mistake_free_wpm_by_keyboard.html')
fig_mistake_free_wpm_by_keyboard



In [17]:
fig_result_hist = px.histogram(
df_trial_results, x = 'WPM', color = 'Keyboard', nbins = 10,
color_discrete_map=keyboard_color_discrete_map)
fig_result_hist.update_layout(
    barmode = 'group')
fig_result_hist.write_html(
analytics_folder+'result_histogram.html')
fig_result_hist

In [18]:
fig_mistake_free_result_hist = px.histogram(
df_mistake_free_trial_results, x = 'WPM', color = 'Keyboard', nbins = 10,
color_discrete_map=keyboard_color_discrete_map)
fig_mistake_free_result_hist.update_layout(
    barmode = 'group')
fig_mistake_free_result_hist.write_html(
analytics_folder+'mistake_free_result_histogram.html')
fig_mistake_free_result_hist

# Part 3: Regression Analyses

Creating categorical variables for different keyboard layouts:

(These variables will be useful for upcoming regression analyses. Also Note that the original 'Keyboard' column is removed in the process of generating these categorical variables.)

In [19]:
df_trial_results_for_regression = pd.get_dummies(
data = df_trial_results, columns = ['Keyboard'], drop_first=True, dtype = 'int').copy() 
# For more on get_dummies(), see: 
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html
df_trial_results

,Test_Number,Unix_Start_Time,Local_Start_Time,UTC_Start_Time,Characters,Seconds,CPS,WPM,Book,Chapter,Verse_Order,Verse,Verse #,Last 10 Avg,Last 100 Avg,Last 1000 Avg,cumulative_avg,Local_Start_Year,Local_Start_Month,Local_Start_Hour,Count,Local_Start_Date,Local_Start_Minute,Local_Start_10_Minute_Block,Local_Start_15_Minute_Block,Local_Start_30_Minute_Block,Local_End_Time,Local_End_Year,Local_End_Month,Local_End_Date,Local_End_Hour,Local_End_Minute,Local_End_15_Minute_Block,Local_End_10_Minute_Block,Local_End_30_Minute_Block,Mistake_Free_Test,WPM_Rank,WPM_Percentile,Backspaces as % of Verse Length,Incorrect Characters as % of Verse Length,Min Character Time (ms),Median Character Time (ms),Max Character Time (ms),Backspace % Last 10 Avg,Backspace % Last 100 Avg,Backspace % Last 1000 Avg,Session,Test_#_Within_Session,Incorrect Character % Last 10 Avg,Incorrect Character % Last 100 Avg,Incorrect Character % Last 1000 Avg,Test Version,Location,Keyboard,Layout,Caffeine,Custom_1,Custom_2,Custom_3,Autostart,% of Words Typed Correctly,Extra Keystrokes %,Extra Keystrokes % Last 10 Avg,Extra Keystrokes % Last 100 Avg,Extra Keystrokes % Last 1000 Avg
9335,9336,1.710171e+09,2024-03-11 11:26:37.472366,2024-03-11 15:26:37.472366+00:00,192,16.778450,11.443250,137.319004,JHN,5,30510,"Jesus therefore answered them, ""Most certainly...",19.0,137.585592,138.500531,128.903824,133.048,2024,3,11,1,2024-03-11,26,3,2,1,2024-03-11 11:26:54.250815700,2024,3,2024-03-11,11,26,2,3,1,0.0,4263,56.349857,3.125000,8.333333,NaN,NaN,NaN,NaN,NaN,NaN,203,1,6.676314,5.521322,7.532080,v2,home,cherry red,dvorak,yes,NaN,NaN,NaN,0.0,88.235294,13.541667,12.194104,NaN,NaN
9336,9337,1.710171e+09,2024-03-11 11:26:56.220335,2024-03-11 15:26:56.220335+00:00,152,11.056986,13.746965,164.963585,JHN,5,30511,"For the Father has affection for the Son, and ...",20.0,140.528065,138.713826,128.928716,133.052,2024,3,11,1,2024-03-11,26,3,2,1,2024-03-11 11:27:07.277320700,2024,3,2024-03-11,11,27,2,3,1,0.0,611,93.752560,1.315789,4.605263,NaN,NaN,NaN,NaN,NaN,NaN,203,2,6.901546,5.540529,7.531622,v2,home,cherry red,dvorak,yes,NaN,NaN,NaN,0.0,96.551724,5.921053,12.315621,NaN,NaN
9337,9338,1.710171e+09,2024-03-11 11:27:11.907570,2024-03-11 15:27:11.907570+00:00,106,7.391912,14.339997,172.079964,JHN,5,30512,For as the Father raises the dead and gives th...,21.0,142.400652,138.888181,128.988686,133.056,2024,3,11,1,2024-03-11,27,3,2,1,2024-03-11 11:27:19.299482300,2024,3,2024-03-11,11,27,2,3,1,1.0,268,97.265465,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,203,3,6.300453,5.540529,7.515985,v2,home,cherry red,dvorak,yes,NaN,NaN,NaN,0.0,100.000000,0.000000,11.168080,NaN,NaN
9338,9339,1.710171e+09,2024-03-11 11:27:21.423686,2024-03-11 15:27:21.423686+00:00,71,6.279210,11.307155,135.685859,JHN,5,30513,"For the Father judges no one, but he has given...",22.0,143.358556,138.959112,128.984768,133.056,2024,3,11,1,2024-03-11,27,3,2,1,2024-03-11 11:27:27.702896000,2024,3,2024-03-11,11,27,2,3,1,0.0,4550,53.410488,1.408451,1.408451,NaN,NaN,NaN,NaN,NaN,NaN,203,4,5.989321,5.501583,7.511974,v2,home,cherry red,dvorak,yes,NaN,NaN,NaN,0.0,93.333333,2.816901,10.376323,NaN,NaN
9339,9340,1.710171e+09,2024-03-11 11:27:31.307316,2024-03-11 15:27:31.307316+00:00,126,10.224435,12.323420,147.881036,JHN,5,30514,"that all may honor the Son, even as they honor...",23.0,147.057733,139.134097,128.998703,133.058,2024,3,11,1,2024-03-11,27,3,2,1,2024-03-11 11:27:41.531750700,2024,3,2024-03-11,11,27,2,3,1,0.0,2442,75.000000,1.587302,4.761905,NaN,NaN,NaN,NaN,NaN,NaN,203,5,5.024833,5.513488,7.514355,v2,home,cherry red,dvorak,yes,NaN,NaN,NaN,0.0,96.000000,9.523810,8.701586,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9759,9760,1.710602e+09,2024-03-16 11:17:41.671556,2024-03-16 15:17:41.671556+00:00,78,6.773794,11.5

The following list allows us to create a list of categorical columns (which will be useful when coding regression analyses) that can get updated automatically if new keyboards are added in.

In [20]:
keyboard_categorical_columns = [
column for column in df_trial_results_for_regression if 'Keyboard_' in column]
keyboard_categorical_columns

['Keyboard_cherry mx2a brown', 'Keyboard_cherry red']

### Regression that evaluates the impact of accuracy on WPM:

In [21]:
y = df_trial_results_for_regression['WPM']
x = df_trial_results_for_regression['Extra Keystrokes %']
x = sm.add_constant(x)
model = sm.OLS(y, x)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    WPM   R-squared:                       0.652
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     448.7
Date:                Sat, 16 Mar 2024   Prob (F-statistic):           8.95e-57
Time:                        15:24:33   Log-Likelihood:                -931.50
No. Observations:                 241   AIC:                             1867.
Df Residuals:                     239   BIC:                             1874.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                159.1113      1.043    152.487      0.000     157.056     161.167
Extra Keystrokes %    -1.6516      0.078    -21.183      0.000      -1.805      -1.498
==============================================================================
Omnibus:                        2.532   Durbin-Watson:                   1.655
Prob(Omnibus):                  0.282   Jarque-Bera (JB):                2.190
Skew:                           0.213   Prob(JB):                        0.335
Kurtosis:                       3.191   Cond. No.                         18.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Regression that evaluates the impact of keyboard choice on WPM:

This regression does not adjust for accuracy, although a later analysis will.

In [22]:
y = df_trial_results_for_regression['WPM']
x = df_trial_results_for_regression[keyboard_categorical_columns]
x = sm.add_constant(x)
model = sm.OLS(y, x)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    WPM   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.9554
Date:                Sat, 16 Mar 2024   Prob (F-statistic):              0.386
Time:                        15:24:33   Log-Likelihood:                -1057.9
No. Observations:                 241   AIC:                             2122.
Df Residuals:                     238   BIC:                             2132.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const                        146.1285      2.194     66.591      0.000     141.806     150.451
Keyboard_cherry mx2a brown    -3.3808      3.094     -1.093      0.276      -9.475       2.714
Keyboard_cherry red           -3.9736      3.103     -1.280      0.202     -10.087       2.140
==============================================================================
Omnibus:                        3.000   Durbin-Watson:                   1.801
Prob(Omnibus):                  0.223   Jarque-Bera (JB):                2.279
Skew:                          -0.081   Prob(JB):                        0.320
Kurtosis:                       2.552   Cond. No.                         3.74
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Regression that evaluates the impact of keyboard choice on accuracy:

In [23]:
y = df_trial_results_for_regression['Extra Keystrokes %']
x = df_trial_results_for_regression[keyboard_categorical_columns]
x = sm.add_constant(x)
model = sm.OLS(y, x)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     Extra Keystrokes %   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     2.333
Date:                Sat, 16 Mar 2024   Prob (F-statistic):             0.0992
Time:                        15:24:33   Log-Likelihood:                -884.14
No. Observations:                 241   AIC:                             1774.
Df Residuals:                     238   BIC:                             1785.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const                          7.8222      1.067      7.330      0.000       5.720       9.924
Keyboard_cherry mx2a brown     3.2311      1.504      2.148      0.033       0.267       6.195
Keyboard_cherry red            1.3223      1.509      0.876      0.382      -1.651       4.295
==============================================================================
Omnibus:                       90.682   Durbin-Watson:                   1.976
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              291.919
Skew:                           1.612   Prob(JB):                     4.08e-64
Kurtosis:                       7.321   Cond. No.                         3.74
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Regression that evaluates the impact of keyboard choice *and* accuracy on WPM:

In [24]:
y = df_trial_results_for_regression['WPM']
x = df_trial_results_for_regression[
['Extra Keystrokes %']+keyboard_categorical_columns]
x = sm.add_constant(x)
model = sm.OLS(y, x)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    WPM   R-squared:                       0.659
Model:                            OLS   Adj. R-squared:                  0.654
Method:                 Least Squares   F-statistic:                     152.4
Date:                Sat, 16 Mar 2024   Prob (F-statistic):           4.87e-55
Time:                        15:24:33   Log-Likelihood:                -929.34
No. Observations:                 241   AIC:                             1867.
Df Residuals:                     237   BIC:                             1881.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const                        159.1560      1.428    111.440      0.000     156.342     161.970
Extra Keystrokes %            -1.6655      0.078    -21.254      0.000      -1.820      -1.511
Keyboard_cherry mx2a brown     2.0005      1.836      1.089      0.277      -1.617       5.618
Keyboard_cherry red           -1.7714      1.827     -0.969      0.333      -5.371       1.828
==============================================================================
Omnibus:                        2.332   Durbin-Watson:                   1.691
Prob(Omnibus):                  0.312   Jarque-Bera (JB):                1.978
Skew:                           0.186   Prob(JB):                        0.372
Kurtosis:                       3.242   Cond. No.                         44.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Regression that evaluates the impact of keyboard choice on WPM for mistake-free results only:

In [25]:
df_mistake_free_trial_results_for_regression = (
df_trial_results_for_regression.query("Mistake_Free_Test == 1").copy())
df_mistake_free_trial_results_for_regression

,Test_Number,Unix_Start_Time,Local_Start_Time,UTC_Start_Time,Characters,Seconds,CPS,WPM,Book,Chapter,Verse_Order,Verse,Verse #,Last 10 Avg,Last 100 Avg,Last 1000 Avg,cumulative_avg,Local_Start_Year,Local_Start_Month,Local_Start_Hour,Count,Local_Start_Date,Local_Start_Minute,Local_Start_10_Minute_Block,Local_Start_15_Minute_Block,Local_Start_30_Minute_Block,Local_End_Time,Local_End_Year,Local_End_Month,Local_End_Date,Local_End_Hour,Local_End_Minute,Local_End_15_Minute_Block,Local_End_10_Minute_Block,Local_End_30_Minute_Block,Mistake_Free_Test,WPM_Rank,WPM_Percentile,Backspaces as % of Verse Length,Incorrect Characters as % of Verse Length,Min Character Time (ms),Median Character Time (ms),Max Character Time (ms),Backspace % Last 10 Avg,Backspace % Last 100 Avg,Backspace % Last 1000 Avg,Session,Test_#_Within_Session,Incorrect Character % Last 10 Avg,Incorrect Character % Last 100 Avg,Incorrect Character % Last 1000 Avg,Test Version,Location,Layout,Caffeine,Custom_1,Custom_2,Custom_3,Autostart,% of Words Typed Correctly,Extra Keystrokes %,Extra Keystrokes % Last 10 Avg,Extra Keystrokes % Last 100 Avg,Extra Keystrokes % Last 1000 Avg,Keyboard_cherry mx2a brown,Keyboard_cherry red
9337,9338,1.710171e+09,2024-03-11 11:27:11.907570,2024-03-11 15:27:11.907570+00:00,106,7.391912,14.339997,172.079964,JHN,5,30512,For as the Father raises the dead and gives th...,21.0,142.400652,138.888181,128.988686,133.056,2024,3,11,1,2024-03-11,27,3,2,1,2024-03-11 11:27:19.299482300,2024,3,2024-03-11,11,27,2,3,1,1.0,268,97.265465,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,203,3,6.300453,5.540529,7.515985,v2,home,dvorak,yes,NaN,NaN,NaN,0.0,100.0,0.0,11.168080,NaN,NaN,0,1
9341,9342,1.710171e+09,2024-03-11 11:27:59.479325,2024-03-11 15:27:59.479325+00:00,132,9.446713,13.973115,167.677375,JHN,5,30516,"Most certainly I tell you, the hour comes, and...",25.0,149.412205,139.781085,129.040766,133.063,2024,3,11,1,2024-03-11,27,3,2,1,2024-03-11 11:28:08.926037800,2024,3,2024-03-11,11,28,2,3,1,1.0,459,95.309299,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,203,7,4.437102,5.319670,7.500135,v2,home,dvorak,yes,NaN,NaN,NaN,0.0,100.0,0.0,7.646606,NaN,NaN,0,1
9344,9345,1.710171e+09,2024-03-11 11:28:28.288209,2024-03-11 15:28:28.288209+00:00,94,6.503514,14.453724,173.444687,JHN,5,30519,"Don't marvel at this, for the hour comes in wh...",28.0,152.617448,139.821860,129.124393,133.069,2024,3,11,1,2024-03-11,28,3,2,1,2024-03-11 11:28:34.791723300,2024,3,2024-03-11,11,28,2,3,1,1.0,214,97.818517,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,203,10,3.013833,5.283769,7.484162,v2,home,dvorak,yes,NaN,NaN,NaN,0.0,100.0,0.0,5.450561,NaN,NaN,0,1
9348,9349,1.710171e+09,2024-03-11 11:30:12.296669,2024-03-11 15:30:12.296669+00:00,100,6.586644,15.182239,182.186865,JHN,5,30523,It is another who testifies about me. I know t...,32.0,148.742679,139.995001,129.153923,133.073,2024,3,11,1,2024-03-11,30,4,3,2,2024-03-11 11:30:18.883312900,2024,3,2024-03-11,11,30,3,4,2,1.0,46,99.539123,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,203,14,3.738254,5.286069,7.460851,v2,home,dvorak,yes,NaN,NaN,NaN,0.0,100.0,0.0,7.207516,NaN,NaN,1,0
9349,9350,1.710171e+09,2024-03-11 11:30:20.082964,2024-03-11 15:30:20.082964+00:00,57,4.147702,13.742551,164.910610,JHN,5,30524,"You have sent to John, and he has testified to...",33.0,150.445636,140.222450,129.177843,133.076,2024,3,11,1,2024-03-11,30,4,3,2,2024-03-11 11:30:24.230665600,2024,3,2024-03-11,11,30,3,4,2,1.0,615,93.711594,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,203,15,3.262063,5.229159,7.456936,v2,home,dvorak,yes,NaN,NaN,NaN,0.0,100.0,0.0,6.255135,NaN,NaN,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,9738,1.710601e+09,2024-03-16 11:03:45.956088,2024-03-16 15:03:45.956088+00:00,60,5.065754,11.844239,142.130865,JHN,9,30733,"Then they asked him, ""Where is he?"" He said, ""...",12.0,141.332413,1

In [26]:
y = df_mistake_free_trial_results_for_regression['WPM']
x = df_mistake_free_trial_results_for_regression[
keyboard_categorical_columns]
x = sm.add_constant(x)
model = sm.OLS(y, x)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    WPM   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.045
Date:                Sat, 16 Mar 2024   Prob (F-statistic):              0.358
Time:                        15:24:34   Log-Likelihood:                -235.01
No. Observations:                  61   AIC:                             476.0
Df Residuals:                      58   BIC:                             482.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const                        167.2069      2.293     72.925      0.000     162.617     171.797
Keyboard_cherry mx2a brown    -2.0361      3.529     -0.577      0.566      -9.099       5.027
Keyboard_cherry red           -5.3710      3.715     -1.446      0.154     -12.807       2.065
==============================================================================
Omnibus:                        3.177   Durbin-Watson:                   1.383
Prob(Omnibus):                  0.204   Jarque-Bera (JB):                2.268
Skew:                          -0.374   Prob(JB):                        0.322
Kurtosis:                       3.577   Cond. No.                         3.39
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [27]:
end_time = time.time()
run_time = end_time - start_time
print(f"Completed script in {round(run_time, 3)} seconds.")

Completed script in 2.034 seconds.
